# Prueba #1

Luis H. Benitez

In [19]:
import pandas as pd

## Recuperando dataset

**Nota importante**
* El siguiente codigo se puede ejecutar como tarea de windows. Se deja aqui para su visualizacion y ejecucion manual

In [20]:
from selenium import webdriver
import time, os
import chromedriver_binary#sin necesidad de añadir a Path
import shutil, os
opciones = webdriver.ChromeOptions() 
prefs = {'download.default_directory' : 'C:\\Users\\luisb\\Code-Font\\prueba\\dataset\\'}

opciones.add_experimental_option('prefs', prefs) 
browser = webdriver.Chrome(options=opciones)

browser.get("https://www.worldbank.org/en/projects-operations/procurement/debarred-firms#")
time.sleep(20)#espera para buscar el boton

#verifico si el archivo existe
if os.path.isfile('C:/Users/luisb/Code-Font/prueba/dataset/Sanctioned individuals and firms.xlsx'):
    print("Eliminando archivo anterior...")
    os.remove('C:/Users/luisb/Code-Font/prueba/dataset/Sanctioned individuals and firms.xlsx')
    #renombro para no borrar y no dañar el siguiente
browser.find_element_by_class_name("k-grid-excel").click()
#time.sleep(5)#deja abierta un momento mientras descarga
#browser.close()

Eliminando archivo anterior...


**NOTA** Este codigo varia ligeramente en el script automatico
- Las rutas del script son explicitas
- se agrega ciclo while para esperar la descarga

## Lectura y limpieza del dataset

In [21]:
time.sleep(6) #para esperar a que acabe la descarga
df = pd.read_excel("./dataset/Sanctioned individuals and firms.xlsx", header=1)
df.rename(columns={'Ineligibility Period': 'Ineligibility Period From Date', 'Unnamed: 5': 'Ineligibility Period To Date'}, inplace=True)
df.head()

,Firm Name,Additional Firm Info,Address,Country,Ineligibility Period From Date,Ineligibility Period To Date,Grounds
0,NaN,NaN,NaN,NaN,From Date,To Date,NaN
1,LLC “WOODMANMEBEL”,*514,"YUROVSKAYA STREET 92, OFFICE NO. 1, MOSCOW 125466",Russian Federation,2019-12-31,2022-12-30,"Procurement Guidelines, 1.16(a)(ii)"
2,ZENG ZILING,NaN,"3-21-01, E-SHIDAI, XIN MAO YUAN, SI GONG LI, C...",China,2019-12-24,2023-09-11,Cross Debarment: ADB
3,GENERAL ELECTRIC POWER SWEDEN AB,NaN,"SLOTTSGATAN 69, SE-602 22 NORRKOPING",Sweden,2019-12-17,2025-11-26,Cross Debarment: EBRD
4,ESWARI ELECTRICALS PVT. LIMITED,*513,"PLOT NO. 64, INDUSTRIAL ESTATE, PERUNGUDI, CHE...",India,2019-11-26,2023-11-25,"Procurement Guidelines, 1.16(a)(ii)"


In [22]:
df.shape

(2084, 7)

In [23]:
df=df.drop(df.index[0])#elimino primera fila (trae headers)
df.head()

,Firm Name,Additional Firm Info,Address,Country,Ineligibility Period From Date,Ineligibility Period To Date,Grounds
1,LLC “WOODMANMEBEL”,*514,"YUROVSKAYA STREET 92, OFFICE NO. 1, MOSCOW 125466",Russian Federation,2019-12-31,2022-12-30,"Procurement Guidelines, 1.16(a)(ii)"
2,ZENG ZILING,NaN,"3-21-01, E-SHIDAI, XIN MAO YUAN, SI GONG LI, C...",China,2019-12-24,2023-09-11,Cross Debarment: ADB
3,GENERAL ELECTRIC POWER SWEDEN AB,NaN,"SLOTTSGATAN 69, SE-602 22 NORRKOPING",Sweden,2019-12-17,2025-11-26,Cross Debarment: EBRD
4,ESWARI ELECTRICALS PVT. LIMITED,*513,"PLOT NO. 64, INDUSTRIAL ESTATE, PERUNGUDI, CHE...",India,2019-11-26,2023-11-25,"Procurement Guidelines, 1.16(a)(ii)"
5,MOTO-ID S.A.,(Reg. No.: B171868)*475,"6, RUE EUGENE RUPPERT, L-2453",Luxembourg,2019-11-22,2025-10-23,"May 2010 Procurement Guidelines, Paras. 1.14(a..."


In [24]:
#conociendo cuantos datos vacios hay en columna Additional Firm Info
pd.isnull(df["Additional Firm Info"]).values.ravel().sum() #deveuvle suma de 1's (true), datos vacios

362

Prefiero no borrar los datos vacios en esta ocasion 

In [25]:
df.dtypes #tipos de dato

Firm Name                         object
Additional Firm Info              object
Address                           object
Country                           object
Ineligibility Period From Date    object
Ineligibility Period To Date      object
Grounds                           object
dtype: object

In [26]:
#reemplazar caracteres que dan conflicto al insertar en la tabla
df['Firm Name'] = df['Firm Name'].str.replace('“','')
df['Firm Name'] = df['Firm Name'].str.replace('”','')
df['Firm Name'] = df['Firm Name'].str.replace("'",' ')
df['Address'] = df['Address'].str.replace("'",' ')
df['Country'] = df['Country'].str.replace("'",' ')
#se dejan los nan 

In [27]:
df.head()

,Firm Name,Additional Firm Info,Address,Country,Ineligibility Period From Date,Ineligibility Period To Date,Grounds
1,LLC WOODMANMEBEL,*514,"YUROVSKAYA STREET 92, OFFICE NO. 1, MOSCOW 125466",Russian Federation,2019-12-31,2022-12-30,"Procurement Guidelines, 1.16(a)(ii)"
2,ZENG ZILING,NaN,"3-21-01, E-SHIDAI, XIN MAO YUAN, SI GONG LI, C...",China,2019-12-24,2023-09-11,Cross Debarment: ADB
3,GENERAL ELECTRIC POWER SWEDEN AB,NaN,"SLOTTSGATAN 69, SE-602 22 NORRKOPING",Sweden,2019-12-17,2025-11-26,Cross Debarment: EBRD
4,ESWARI ELECTRICALS PVT. LIMITED,*513,"PLOT NO. 64, INDUSTRIAL ESTATE, PERUNGUDI, CHE...",India,2019-11-26,2023-11-25,"Procurement Guidelines, 1.16(a)(ii)"
5,MOTO-ID S.A.,(Reg. No.: B171868)*475,"6, RUE EUGENE RUPPERT, L-2453",Luxembourg,2019-11-22,2025-10-23,"May 2010 Procurement Guidelines, Paras. 1.14(a..."


In [28]:
#df['Ineligibility Period From Date'] =  pd.to_datetime(df['Ineligibility Period From Date'])

## Guardando en DataBase

In [29]:
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.types import String
import io
con = psycopg2.connect(database="prueba", user="postgres", password="12345", host="127.0.0.1", port="5432")

In [30]:
cur = con.cursor()

In [31]:
cur.execute('''CREATE TABLE IF NOT EXISTS diccionario
(
    "firmname" TEXT,
    "additionalfirminfo" TEXT,
    "adress" TEXT,
    "country" TEXT,
    "ineligibilityperiodfromdate" TEXT,
    "ineligibilityperiodtodate" TEXT,
    "grounds" TEXT
);''')
print("Tabla creada")
con.commit()
con.close()

Tabla creada


In [32]:
con = psycopg2.connect(database="prueba", user="postgres", password="12345", host="127.0.0.1", port="5432")
cur= con.cursor()
i=1
num= df.shape[0]
while i <= num:
    consulta = "INSERT INTO diccionario(firmname, additionalfirminfo,adress,country,ineligibilityperiodfromdate,ineligibilityperiodtodate,grounds) VALUES ('{}', '{}', '{}', '{}', '{}', '{}', '{}')".format(
        df['Firm Name'][i],df['Additional Firm Info'][i], df['Address'][i],df['Country'][i],
        df['Ineligibility Period From Date'][i],df['Ineligibility Period To Date'][i], df['Grounds'][i])
    cur.execute(consulta);
    con.commit()
    i+=1
    
con.close()

el siguiente codigo dio multiples problemas. Sigo buscando una alternativa mas rapida para evitar el llenado escribiendo los campos explicitamente
```python
obj_cols = df.select_dtypes(include=[object]).columns.values.tolist()
df.to_sql('diccionario', con, if_exists='append')

df.to_sql(name='new_table',con=con,if_exists='append')
con.close()
```